Alex-Net（2012）
引起此次深度学习热潮的第一个网络
此前更为广泛应用的是核方法（支持向量机SVM）2000年左右，SVM为凸优化模型，模型理论完备
Alex-Net是更粗更深的Le-Net,11-11的卷积核替换5-5的卷积核，3-3的最大池化替换2-2的平均池化，relu激活替换sig激活（relu激活能够避免Loss梯度消失问题，在零处导数为1），并且添加了几个卷积层，使用了丢弃法和数据增强。针对的数据集也从简单的黑白图像50000-10000（Fashion-MNIST）或手写数据集转换为RGB的复杂现实图像Image-Net

le-Net和Alex-Net都是卷积层与池化层组合形成的，VGG网络则是将多个卷积（3*3 padding=1,stride=1）和最大池化（3*3 padding=2）（宽度减半）作为VGG块，多个VGG块组成VGG网络，VGG-16代表卷积层个数。
le-Net Alex-Net VGG最终都需要全连接层，导致最终参数数目过大（与像素数有关），很容易过拟合，因此出现了一种不需要全连接层的网络，NiN(网络中的网络)，NiN用一个卷积层加两个1*1的卷积层组成块，多个块+最大池化组成网络，最终用全局池化输出类。1*1的卷积相当于对输入输出通道每个像素点做全连接，大大减小了参数数目，模型简单且不容易过拟合。

In [1]:
%matplotlib 

Using matplotlib backend: TkAgg


In [2]:
import torch

In [3]:
import torch.nn as nn

In [4]:
import torch.utils.data as data

In [5]:
import torchvision

In [6]:
import matplotlib.pyplot as pyp

In [7]:
import torchvision.transforms as trans

In [8]:
import numpy as np

In [9]:
import matplotlib.animation as animation

In [10]:
import scipy

In [11]:
transformCom=trans.Compose([trans.Resize(224),trans.ToTensor()])

In [12]:
trainSet=torchvision.datasets.FashionMNIST('../',train=True,download=False,transform=transformCom)
testSet=torchvision.datasets.FashionMNIST('../',train=False,download=False,transform=transformCom)

In [13]:
trainLoader=data.DataLoader(trainSet,batch_size=128,shuffle=True)
testLoader=data.DataLoader(testSet,batch_size=128,shuffle=True)

In [14]:
#AlexeNET网络，relu激活
AlexNet=torch.nn.Sequential(
      nn.Conv2d(1,96,kernel_size=11,stride=4,padding=2),nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2), # 
      nn.Conv2d(96,128*2,kernel_size=5,padding=2),nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2),
      nn.Conv2d(128*2,192*2,kernel_size=3,padding=1),nn.ReLU(),
      nn.Conv2d(192*2,192*2,kernel_size=3,padding=1),nn.ReLU(),
      nn.Conv2d(192*2,256,kernel_size=3,padding=1),nn.ReLU(),
      nn.MaxPool2d(kernel_size=3,stride=2), # 6*6*256
      nn.Flatten(),
      nn.Linear(6*6*256,2048*2),nn.ReLU(),nn.Dropout(p=0.5),
      nn.Linear(2048*2,2048*2),nn.ReLU(),nn.Dropout(p=0.5),#全连接层通常相比卷积层有巨量的参数，非常容易过拟合，因此需要dropout惩罚，防止过拟合，相当于减小了一半的参数数量
      nn.Linear(2048*2,10),nn.ReLU(),
    )

In [15]:
torch.cuda.is_available()

True

In [16]:
AlexNet.cuda(0)

Sequential(
  (0): Conv2d(1, 96, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (1): ReLU()
  (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (3): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (4): ReLU()
  (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (6): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (7): ReLU()
  (8): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): ReLU()
  (10): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU()
  (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (13): Flatten(start_dim=1, end_dim=-1)
  (14): Linear(in_features=9216, out_features=4096, bias=True)
  (15): ReLU()
  (16): Dropout(p=0.5, inplace=False)
  (17): Linear(in_features=4096, out_features=4096, bias=True)
  (18): ReLU()
  (19): Dropout(p=0.5, inplace=False)
  (20): Linear(in_featu

In [17]:
optim=torch.optim.SGD(AlexNet.parameters(),lr=0.01)
loss=torch.nn.CrossEntropyLoss()
loss.cuda()

CrossEntropyLoss()

In [18]:
#创建曲线图并初始化
xdata=[]
ydata=[]
xdataTest=[]
ydataTest=[]
fig, ax = pyp.subplots()
line, = ax.plot(xdata, ydata,color='blue')
line2, = ax.plot(xdataTest, ydataTest,color='red')
pyp.title("Loss")
pyp.xlabel("epoch")
pyp.ylabel("Loss")
pyp.grid()
ax.legend((line,line2),('trainLoss','testLoss'))
#动态更新曲线训练函数
num_epoch=22
aveLoss=torch.zeros(num_epoch)
aveLoss=aveLoss.detach().numpy()
aveLossTest=torch.zeros(num_epoch)
# aveLossTest=aveLoss.detach().numpy()
def animationTrain(epoch):
    #每epoch训练代码区#
    sumLoss=0
    trainnum=0;
    if epoch>0:
        for trainData,trainLabel in trainLoader:
            optim.zero_grad()
            trainLabel=trainLabel.to(torch.device("cuda:0"))
            trainData=trainData.to(torch.device("cuda:0"))
            comLoss=loss(AlexNet(trainData),trainLabel)
            comLoss.backward()
            optim.step()
            sumLoss=comLoss+sumLoss
            trainnum=trainnum+1
        aveLoss[epoch-1]=(sumLoss.to(torch.device("cpu"))/trainnum)

        sumLoss=0
        trainnum=0
        with torch.no_grad():
            for trainData,trainLabel in testLoader:
                trainLabel=trainLabel.to(torch.device("cuda:0"))
                trainData=trainData.to(torch.device("cuda:0"))
                comLoss=loss(AlexNet(trainData),trainLabel)
                sumLoss=comLoss+sumLoss
                trainnum=trainnum+1
        aveLossTest[epoch-1]=(sumLoss.to(torch.device("cpu"))/trainnum)   
        print(aveLossTest)
        #图表数据更新#
        xdata.append(epoch-1)
        ydata.append(aveLoss[epoch-1])
        xdataTest.append(epoch-1)
        ydataTest.append(aveLossTest[epoch-1])
        line.set_xdata(xdata)
        line.set_ydata(ydata)
        line2.set_xdata(xdataTest)
        line2.set_ydata(ydataTest)
        ax.set_xlim(0, max(xdata))
        ax.set_ylim(0, max(ydata))
        #print(aveLoss[epoch])
        return line,line2,
#创建动画对象并显示，显示过程逐次调用Trian函数
anim = animation.FuncAnimation(fig, animationTrain, interval=20, blit=False,repeat=False,frames=num_epoch)
pyp.show()     


tensor([2.2996, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000])


/tmp/ipykernel_5355/706273932.py:56: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  ax.set_xlim(0, max(xdata))


tensor([2.2996, 1.2494, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000])
tensor([2.2996, 1.2494, 0.9125, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000])
tensor([2.2996, 1.2494, 0.9125, 0.7919, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000])
tensor([2.2996, 1.2494, 0.9125, 0.7919, 0.6440, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000])
tensor([2.2996, 1.2494, 0.9125, 0.7919, 0.6440, 0.5759, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.00